In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import ndcg_score

import pickle

In [2]:
training_data = pd.read_csv('data/training_set_VU_DM.csv')

In [3]:
training_data['target'] = (5*training_data['booking_bool']) + training_data['click_bool']

In [4]:
z = training_data['prop_id'].value_counts() 
dict_z = z.to_dict() # converts to dictionary
training_data['counts'] = training_data['prop_id'].map(dict_z)

In [5]:
training_data['prop_loc_score1_norm'] = training_data['prop_location_score1'] / training_data['prop_location_score1'].abs().max()
training_data['prop_location_score2'].fillna(training_data['prop_loc_score1_norm'], inplace=True)
training_data['location_score'] = (training_data['prop_loc_score1_norm'] + training_data['prop_location_score2'])/2
training_data = training_data.drop('prop_loc_score1_norm', axis=1)

In [6]:
training_data

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,target,counts,location_score
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,0.0,0.0,NaN,0,NaN,0,0,612,0.224622
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,0.0,0.0,NaN,0,NaN,0,0,583,0.165043
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,0.0,0.0,NaN,0,NaN,0,0,551,0.169843
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,-1.0,0.0,5.0,0,NaN,0,0,460,0.208972
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,0.0,0.0,NaN,0,NaN,0,0,665,0.251162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,77700,3,4.0,...,NaN,NaN,NaN,NaN,0,NaN,0,0,47,0.138880
4958343,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,88083,3,4.0,...,NaN,NaN,NaN,NaN,0,NaN,0,0,38,0.215685
4958344,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,94508,3,3.5,...,NaN,NaN,NaN,NaN,0,NaN,0,0,43,0.086997
4958345,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,128360,3,5.0,...,NaN,NaN,NaN,NaN,1,157.84,1,6,37,0.172785


In [37]:
model_traindata = training_data[['srch_id',
                                   'site_id',
                                   'visitor_location_country_id',
                                   'prop_country_id',
                                   'prop_id',
                                   'prop_starrating',
                                   'prop_review_score',
                                   'prop_brand_bool',
#                                    'prop_location_score1',
#                                    'prop_location_score2',
                                   'prop_log_historical_price',
                                   'price_usd',
                                   'promotion_flag',
                                   'srch_destination_id',
                                   'srch_length_of_stay',
                                   'srch_booking_window',
                                   'srch_adults_count',
                                   'srch_children_count',
                                   'srch_room_count',
                                   'srch_saturday_night_bool',
                                   'orig_destination_distance',
                                   'random_bool',
                                   'target',
                                   'counts',
                                   'location_score'
                                  ]]

In [16]:
# model_traindata = model_traindata.dropna().reset_index(drop=True)
# model_traindata = model_traindata.fillna(0)

In [17]:
model_traindata

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_log_historical_price,price_usd,...,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,target,counts,location_score
0,1,12,187,219,893,3,3.5,1,4.95,104.77,...,0,4,0,1,1,0.00,1,0,612,0.224622
1,1,12,187,219,10404,4,4.0,1,5.03,170.74,...,0,4,0,1,1,0.00,1,0,583,0.165043
2,1,12,187,219,21315,3,4.5,1,4.92,179.80,...,0,4,0,1,1,0.00,1,0,551,0.169843
3,1,12,187,219,27348,2,4.0,1,4.39,602.77,...,0,4,0,1,1,0.00,1,0,460,0.208972
4,1,12,187,219,29604,4,3.5,1,4.93,143.58,...,0,4,0,1,1,0.00,1,0,665,0.251162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,332785,5,219,219,77700,3,4.0,1,0.00,118.00,...,21,3,0,1,0,550.92,0,0,47,0.138880
4958343,332785,5,219,219,88083,3,4.0,1,0.00,89.00,...,21,3,0,1,0,553.14,0,0,38,0.215685
4958344,332785,5,219,219,94508,3,3.5,1,0.00,99.00,...,21,3,0,1,0,544.43,0,0,43,0.086997
4958345,332785,5,219,219,128360,3,5.0,1,0.00,139.00,...,21,3,0,1,0,550.38,0,6,37,0.172785


In [38]:
gss = GroupShuffleSplit(test_size=.40, n_splits=1, random_state = 7).split(model_traindata, groups=model_traindata['srch_id'])

X_train_inds, X_test_inds = next(gss)

train_data= model_traindata.iloc[X_train_inds]
X_train = train_data.loc[:, ~train_data.columns.isin(['srch_id','target'])]
y_train = train_data.loc[:, train_data.columns.isin(['target'])]

groups = train_data.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

test_data= model_traindata.iloc[X_test_inds]

# We need to keep the id for later predictions
X_test = test_data.loc[:, ~test_data.columns.isin(['target'])]
y_test = test_data.loc[:, test_data.columns.isin(['target'])]

In [ ]:
model = xgb.XGBRanker(  
    verbosity=2,
    tree_method='hist',
    booster='dart',
    objective='rank:ndcg',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75 
    )

model.fit(X_train, y_train, group=groups, verbose=True)

In [ ]:
filename = 'XGB_model_rankndcg_dart_locscore_nans.sav'
pickle.dump(model, open(filename, 'wb'))

In [31]:
def predict(model, df):
    return model.predict(df.loc[:, ~df.columns.isin(['srch_id'])])
  
predictions = (X_test.groupby('srch_id').apply(lambda x: predict(model, x)))

In [33]:
y_pred = []

for p in predictions:
    p = list(p)
    y_pred.extend(p)

In [34]:
X_test["predicted_ranking"] = y_pred

In [35]:
X_test

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_log_historical_price,price_usd,...,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,counts,location_score,predicted_ranking
65,8,5,219,219,10250,3,3.5,1,4.64,82.00,...,0,1,0,1,0,5.40,0,186,0.115620,-0.153121
66,8,5,219,219,13252,4,4.5,1,5.49,160.00,...,0,1,0,1,0,3.98,0,251,0.000000,0.523100
67,8,5,219,219,22756,4,4.0,1,5.11,259.00,...,0,1,0,1,0,4.41,0,184,0.315272,-0.612062
68,8,5,219,219,27669,3,3.5,1,4.62,85.00,...,0,1,0,1,0,4.17,0,195,0.320250,0.551191
69,8,5,219,219,30630,3,4.0,1,4.81,109.00,...,0,1,0,1,0,3.69,0,218,0.000000,-0.662207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958336,332784,24,219,100,114177,4,4.5,1,0.00,300.74,...,6,1,0,1,0,328.90,0,526,0.385708,-0.787483
4958337,332784,24,219,100,115562,4,4.0,1,0.00,213.15,...,6,1,0,1,0,328.83,0,281,0.374092,0.059043
4958338,332784,24,219,100,121275,4,4.0,0,0.00,193.87,...,6,1,0,1,0,329.68,0,406,0.375984,0.784071
4958339,332784,24,219,100,128677,3,3.5,1,0.00,107.05,...,6,1,0,1,0,328.72,0,379,0.400398,0.455284


In [36]:
print("--> NDCG : ", ndcg_score([y_test['target']],[X_test['predicted_ranking']]))

--> NDCG :  0.768692374404902


**No further feature engineering, including these features**

model_traindata = training_data[['srch_id',
                                   'site_id',
                                   'visitor_location_country_id',
                                   'prop_country_id',
                                   'prop_id',
                                   'prop_starrating',
                                   'prop_review_score',
                                   'prop_brand_bool',
                                   'prop_location_score1',
                                   'prop_location_score2',
                                   'prop_log_historical_price',
                                   'price_usd',
                                   'promotion_flag',
                                   'srch_destination_id',
                                   'srch_length_of_stay',
                                   'srch_booking_window',
                                   'srch_adults_count',
                                   'srch_children_count',
                                   'srch_room_count',
                                   'srch_saturday_night_bool',
                                   'orig_destination_distance',
                                   'random_bool',
                                   'target',
                                   'counts',
                                  ]]

NDCG score on training data XGBoost with objective `rank:ndcg`, booster `gbtree`, dropped NaNs, no further feature engineering = **0.7793220497613718**

*model = xgb.XGBRanker(  
    tree_method='hist',
    booster='gbtree',
    objective='rank:ndcg',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75 
    )*
    
NDCG score on training data XGBoost with objective `rank:ndcg`, booster `dart`, dropped NaNs, no further feature engineering = **0.7798114072823272**


*model = xgb.XGBRanker(  
    tree_method='hist',
    booster='dart',
    objective='rank:ndcg',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75 
    )*
    
NDCG score on training data XGBoost with objective `rank:ndcg`, booster `gbtree`, including NaNs, merge loc_scores = **0.7687506555406398**


*model = xgb.XGBRanker(  
    tree_method='hist',
    booster='gbtree',
    objective='rank:ndcg',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75 
    )*
    
NDCG score on training data XGBoost with objective `rank:ndcg`, booster `gbtree`, replacing NaNs with zeroes, merge loc_scores = **0.7686738092959273**


*model = xgb.XGBRanker(  
    tree_method='hist',
    booster='gbtree',
    objective='rank:ndcg',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75 
    )*
        
NDCG score on training data XGBoost with objective `rank:ndcg`, booster `dart`, including NaNs, merge loc_scores = **?**


*model = xgb.XGBRanker(  
    tree_method='hist',
    booster='dart',
    objective='rank:ndcg',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75 
    )*
    
NDCG score on training data XGBoost with objective `rank:ndcg`, booster `dart`, replacing NaNs with zeroes, merge loc_scores = **0.768692374404902**


*model = xgb.XGBRanker(  
    tree_method='hist',
    booster='dart',
    objective='rank:ndcg',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75 
    )*

In [66]:
test_data = pd.read_csv('data/test_set_VU_DM.csv')

In [67]:
z = test_data['prop_id'].value_counts() 
dict_z = z.to_dict() # converts to dictionary
test_data['counts'] = test_data['prop_id'].map(dict_z)

In [99]:
model_testdata = test_data[['srch_id',
                            'site_id',
                            'visitor_location_country_id',
                            'prop_country_id',
                            'prop_id',
                            'prop_starrating',
                            'prop_review_score',
                            'prop_brand_bool',
                            'prop_location_score1',
                            'prop_location_score2',
                            'prop_log_historical_price',
                            'price_usd',
                            'promotion_flag',
                            'srch_destination_id',
                            'srch_length_of_stay',
                            'srch_booking_window',
                            'srch_adults_count',
                            'srch_children_count',
                            'srch_room_count',
                            'srch_saturday_night_bool',
                            'orig_destination_distance',
                            'random_bool',
                            'counts'
                           ]]

In [100]:
def predict(model, df):
    return model.predict(df.loc[:, ~df.columns.isin(['srch_id'])])
  
test_predictions = (model_testdata.groupby('srch_id').apply(lambda x: predict(model, x)))

In [101]:
test_predictions

srch_id
1         [0.1625433, 0.41950408, -0.13107207, -0.340134...
3         [1.3109548, 1.5259281, 0.08382395, 0.7322764, ...
6         [0.12061024, 0.30068672, 0.3082502, 0.02565842...
7         [-1.4230359, -0.85277504, -0.3971758, -0.90931...
10        [1.468401, 1.2862945, -0.93367916, 1.6723679, ...
                                ...                        
332781    [-0.49719226, -1.1120768, -0.60463244, -1.2823...
332783    [-1.3466609, 1.7371362, 0.5355207, -0.01810777...
332785    [0.70702094, -0.8647833, 1.3359388, 0.29851568...
332786    [-0.3384254, -0.44028246, -0.36365402, 0.77885...
332787    [1.7464826, 0.9119617, 0.38222784, 1.41004, 1....
Length: 199549, dtype: object

In [102]:
y_testpred = []

for p in test_predictions:
    p = list(p)
    y_testpred.extend(p)

In [103]:
model_testdata["predicted_ranking"] = y_testpred

/var/folders/zb/qwtwsp1d539dg2nj7t84r7mc0000gn/T/ipykernel_36153/2849844749.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_testdata["predicted_ranking"] = y_testpred


In [104]:
model_testdata = model_testdata.sort_values(["srch_id","predicted_ranking"], ascending=[True,False])

In [105]:
model_testdata.drop(model_testdata.columns.difference(['srch_id','prop_id']), 1, inplace=True)

/var/folders/zb/qwtwsp1d539dg2nj7t84r7mc0000gn/T/ipykernel_36153/542941893.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  model_testdata.drop(model_testdata.columns.difference(['srch_id','prop_id']), 1, inplace=True)


In [106]:
model_testdata.to_csv("output-XGB-dart.csv", index=False) 